In [12]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [13]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [14]:
import chennai_config
import loaders.cognicity_loader as c

config = chennai_config.config
loader = c.CognicityLoader(config)

2019-08-05 07:36:19,238 - DEBUG - CognicityLoader constructed


In [15]:
text_data = loader.get_data()
import textwrap
def wr(inp):
    res = ""
    sp = textwrap.wrap(inp, 50)
    for each in sp:
        res += each + "\n"
    return res

d = {"text": wr}

s = text_data.loc[text_data.index > 168].loc[text_data.index < 178].to_latex(col_space=20, formatters=d)
# print(s)
# text_data.loc[text_data.index > 168].loc[text_data.index < 178]

In [16]:
# import bow as bow
# bow_labeler = bow.bow_labeler(config)

In [17]:
# import image_recognition.perceptron as ml
# # def run_bigram(report_texts):
# #     vocab, occur, reports_dict =  make_bigram_vocab(report_texts)
# #     features_w_pkey, labels = make_feature_matrix(vocab, reports_dict, flood, no_flood, make_bigram_feature_vector)
# #     features = features_w_pkey[1:, :]
# #     return vocab, occur, reports_dict, features_w_pkey, features, labels
#     
# def run_bow(report_texts):
#     vocab, occur, reports_dict =  bow_labeler.make_unary_vocab(report_texts)
#     features_w_pkey, labels = bow_labeler.make_feature_matrix(reports_dict, vocab)
#     features = features_w_pkey[1:, :]
#     return vocab, occur, reports_dict, features_w_pkey, features, labels
# 
# bow_vocab, bow_occur, bow_reports_dict, bow_features_w_pkey, bow_features, bow_labels = run_bow(loader.get_data())
# 
# print(bow_vocab)
# 
# vocab, occur, reports_dict, features_w_pkey, features, labels = run_bigram(loader.get_data())
# print(vocab)
# 
# features = np.vstack((features, bow_features))
# 
# # update the vocab to have correct index in the new feat. vector
# i = len(vocab)
# for key, value in bow_vocab.items():
#     vocab[key] = value + i
#     occur[key] = bow_occur[key]

# th, th0 = ml.perceptron(features, labels, params={"T":1000, "print":True})

In [18]:
import numpy as np

def prepare_text(inp):
    '''
    returns a list of strings where each string is a different token
    '''
    # TODO replace this with regex?
    import string
    exclude = set(string.punctuation)
    s = "".join(ch for ch in inp if ch not in exclude )
    return s.lower().replace('\n', ' ').split()
    
def make_unary_vocab(pkey_text_df, offset=0):
    """
        Params:
            pkey_text_df: a pandas DataFrame with an index of pkey and a "text" column
            offset: where to begin indexing
        Returns:
            vocab(string: integer) - a dictionary from a word to an integer index
            occur(string: integer) - dictionary from word to the number of times it occurs in the corpus
            reports_dict(pkey: text)  - dictionary from pkey to report text
    """
    # go through all report texts creating set
    # of all possible words
    vocab = dict()
    occur = dict()
    index = offset
    reports_dict = dict()
    for row in pkey_text_df.iterrows():
        report_text_list = prepare_text(row[1]['text'])
        pkey = row[0]
        reports_dict[pkey] = report_text_list
        for word in report_text_list:
            if word not in vocab:
                vocab[word] = index
                index += 1
            if word not in occur:
                occur[word] = 1
            else:
                occur[word] += 1
    return (vocab, occur, reports_dict)

            
def make_bigram_vocab(pkey_text_df, offset=0):
    """
    """
    vocab = dict()
    occur = dict()
    reports_dict = dict()
    index = offset
    for row in pkey_text_df.iterrows():
        report_text_list = prepare_text(row[1]['text'])
        pkey = row[0]
        two_gram = list(zip(report_text_list, report_text_list[1:]))
        reports_dict[pkey] = two_gram
        for first, second in two_gram:
            token = (first, second)
            if token not in vocab:
                vocab[token] = index
                index += 1
            if token in occur:
                occur[token] += 1
            else:
                occur[token] = 1
        
    return (vocab, occur, reports_dict)

            
def make_unary_feature_vector(vocab, report_text_list):
    res = np.zeros((len(vocab), 1))
    for word in report_text_list:
        if word in vocab:
            res[vocab[word]][0] = 1
        # how to deal with out of vocab words?
    return res

def make_bigram_feature_vector(vocab, report_text_list):
    # needs a tuple vocab[(tup)] -> index
    res = np.zeros((len(vocab), 1))
    for first, second in report_text_list:
        if (first, second) in vocab:
            res[vocab[(first, second)]][0] = 1
        # how to deal with out of vocab words?
    return res
        

def make_feature_matrix(vocab, reports_dict, pos_pkeys, neg_pkeys, make_vector=make_unary_feature_vector):
    """
    params:
        vocab (dict): dictionary from word to index location along the
                  column vector
        reports_dict: dictionary from pkeys to text
        pos_pkeys (set): membership in this set means the pkey should be 
                         labeled as postive
        make_vector: a function that takes a vocab and report_text_list and returns a numpy feature vector 
        
    Returns:
        tuple of (feature_matrix, labels)
        feature_matrix is a matrix of pkeys with associated feature columns underneath
        pkey      | pkey
        feat_vect | feat
        size: (len(vocab)+1, num_reports)
        labels is a matrix of size (1, num_reports) where report i is +1 if 
            pkey matching that index is in pos_pkeys and -1 else
    """
    
    labels = np.zeros((1, (len(reports_dict))))
    feature_matrix = np.zeros((len(vocab)+1, len(reports_dict)))
    i = 0
    for pkey, word_list in reports_dict.items():
        col = make_vector(vocab, word_list)
        col_w_pkey = np.insert(col, 0, [pkey], 0)
        feature_matrix[:, i] = col_w_pkey[:,0]
        if pkey in pos_pkeys:
            labels[0, i] = 1
        else: 
            labels[0, i] = -1
        i += 1
    return (feature_matrix, labels)

In [19]:
import pandas as pd
start_known_flood = "'2017-11-01 00:00:35.630000+05:30'" 
end_known_flood = "'2017-11-07 00:00:35.630000+05:30'"

def __get_flood_pkeys(start_date, end_date, engine):
    # gets the pkeys of reports during flood dates
    from sqlalchemy import text

    pkeys = pd.read_sql_query('''
        SELECT pkey, created_at FROM ''' + config['database_name'] + '''.all_reports WHERE
            created_at > %(start_date)s::timestamptz
                AND 
            created_at < %(end_date)s::timestamptz
    ''', params={"start_date": start_date, "end_date": end_date}, con=engine, index_col="pkey")
    
    return set(pkeys.index)
    
def __get_no_flood_pkeys(start_flood_date, end_flood_date, engine):
    # gets the pkeys of reports outside dates
    from sqlalchemy import text

    pkeys = pd.read_sql_query('''
        SELECT pkey, created_at FROM ''' + config['database_name'] + '''.all_reports WHERE
            created_at < %(start_date)s::timestamptz
                OR 
            created_at > %(end_date)s::timestamptz
    ''', params={"start_date": start_flood_date, "end_date": end_flood_date}, con=engine, index_col="pkey")

    return set(pkeys.index)

flood = __get_flood_pkeys(start_known_flood, end_known_flood, config['database_engine'])
no_flood = __get_no_flood_pkeys(start_known_flood, end_known_flood, config['database_engine'])
percent = len(flood)/(len(flood)+len(no_flood))
print("percent of flooding: ", percent)


percent of flooding:  0.75


In [20]:


import image_recognition.perceptron as ml
def run_bigram(report_texts):
    vocab, occur, reports_dict =  make_bigram_vocab(report_texts)
    features_w_pkey, labels = make_feature_matrix(vocab, reports_dict, flood, no_flood, make_bigram_feature_vector)
    features = features_w_pkey[1:, :]
    return vocab, occur, reports_dict, features_w_pkey, features, labels
    
def run_bow(report_texts):
    vocab, occur, reports_dict =  make_unary_vocab(report_texts)
    features_w_pkey, labels = make_feature_matrix(vocab, reports_dict, flood, no_flood, make_unary_feature_vector)
    features = features_w_pkey[1:, :]
    return vocab, occur, reports_dict, features_w_pkey, features, labels

bow_vocab, bow_occur, bow_reports_dict, bow_features_w_pkey, bow_features, bow_labels = run_bow(loader.get_data())

print(bow_vocab)

vocab, occur, reports_dict, features_w_pkey, features, labels = run_bigram(loader.get_data())
print(vocab)

features = np.vstack((features, bow_features))

# update the vocab to have correct index in the new feat. vector
i = len(vocab)
for key, value in bow_vocab.items():
    vocab[key] = value + i
    occur[key] = bow_occur[key]

th, th0 = ml.perceptron(features, labels, params={"T":1000, "print":True})

{'floody': 903, 'condition': 911, 'sozhanagar': 774, 'below': 108, 'thevar': 552, 'if': 143, 'shing': 793, 'cars': 184, 'vidhyashram': 515, 'pot': 832, 'between': 162, 'konnur': 578, 'stagnated': 9, 'walk': 88, 'turned': 620, '11th': 297, 'dhandesswaram': 325, 'friday': 554, 'hardly': 629, 'comes': 645, 'underpass': 724, 'network': 288, 'switch': 247, 'interior': 504, 'arcot': 158, 'garbage': 706, 'wastewater': 610, 'bike': 433, 'quick': 752, 'right': 520, 'pallikaranai': 178, 'lying': 485, 'live': 856, 'thigh': 521, 'known': 240, 'picture': 29, 'mosquitoes': 875, 'hope': 68, 'falan': 7, 'kuberan': 384, 'year': 339, 'ride': 432, 'levels': 407, 'spreading': 899, 'hospitals': 881, 'intrution': 156, '2feet': 292, 'government': 778, 'than': 349, 'shastri': 296, 'drkalaingar': 924, 'that': 20, 'luz': 74, 'ags': 317, 'inspite': 665, 'hasthinapuramchromepet': 809, 'draining': 279, 'higher': 399, 'ra': 675, 'raman': 615, 'when': 730, 'enter': 113, 'stagnatted': 910, 'in': 59, 'good': 725, 'kal

T:  200
score:  [[0.97560976]]
T:  400
score:  [[0.97560976]]
T:  600
score:  [[0.97560976]]
T:  800
score:  [[0.97560976]]


In [26]:
def random_cross_validate_model(data, labels, learner=ml.perceptron, k=5):
    full = np.vstack((data,labels)).copy()
    np.random.shuffle(full.T)
    data = full[:-1, :]
    labels = full[-1:, :]

    # axis=1 means columns, split into groups of cols
    s_data = np.array_split(data, k, axis=1)
    s_labels = np.array_split(labels, k, axis=1)
    score_sum = 0
    for i in range(k):
        data_train = np.concatenate(s_data[:i] + s_data[i+1:], axis=1)
        labels_train = np.concatenate(s_labels[:i] + s_labels[i+1:], axis=1)
        data_test = np.array(s_data[i])
        labels_test = np.array(s_labels[i])
        
        th, th0 = learner(data_train, labels_train, params={"T":1000, 'print':False})

        # number correctly predicted in test set
        score = ml.score(data_test, labels_test, th, th0)
        
        score_sum += score
    return (score_sum)/(labels.shape[1]), score_sum

def score_bow():
    bow_vocab, bow_occur, bow_reports_dict, bow_features_w_pkey, bow_features, bow_labels = run_bow(loader.get_data())
    
    print(len(bow_vocab))
    score, raw = random_cross_validate_model(bow_features, bow_labels, k=5)
    print(score)
    return bow_features
    

def score_bigram():
    vocab, occur, reports_dict, features_w_pkey, features, labels = run_bigram(loader.get_data())
    print(len(vocab))
    score, raw = random_cross_validate_model(features, labels, k=5)
    print(score)

print("bow")
score_bow()

print("bigram")
score_bigram()


print("both")
score, raw = random_cross_validate_model(features, labels, k=5)
print("instant: ", score)

#overall = 0
#for i in range(1,5):
#    score, raw = cross_validate_model(features, labels, k=7)
#    overall += raw
#    print("instant: ", score)
#    print(overall/(i*labels.shape[1]))

bow
933
[[0.71341463]]
bigram
2214
[[0.53658537]]
both
instant:  [[0.74390244]]


In [ ]:
chennai = pd.read_sql_query('''
    SELECT pkey, created_at, text, image_url, disaster_type, report_data, tags FROM ''' + config["database_name"] + '''.all_reports 
''', params={"start_date": start_known_flood, "end_date": end_known_flood}, con=config["database_engine"], index_col="pkey")
pd.options.display.max_rows = None
pd.options.display.max_colwidth = 1000
#from IPython.display import display
#chennennaii

In [10]:
res = np.dot(features.T, th) + th0
predicted_flood = (res > 0).T[0,:]

preds = np.vstack((features_w_pkey[0,:], predicted_flood)).T
preds

pred_w_pkey = pd.DataFrame(data=np.bool_(preds[:,1]), index=np.int_(preds[:,0]), columns=['predicted_flood'])
total_data = pd.concat([chennai,pred_w_pkey], axis=1, join='inner')

def f(inp):
    return inp.get('flood_depth', None)

total_data['flood_depth'] = total_data.report_data.apply(f)
# total_data['flood_depth'] =  pd.io.json.json_normalize(total_data.report_data, meta='flood_depth')['flood_depth']

total_data.drop(['tags'], axis=1)
# total_data['created_at'] = pd.to_datetime(total_data['created_at'], utc=True)
# total_data['created_at'] = pd.to_datetime(total_data['created_at'])
total_data

NameError: name 'chennai' is not defined

In [9]:
%matplotlib widget

In [ ]:
#total_data = total_data.set_index(['created_at'], append=True, drop=False)
#plt.plot(total_data['created_at'], total_data['flood_depth'])
#total_data.loc[total_data.index < 400, :]
plot_data = total_data.loc[total_data.index < 600, :].copy()

# let's color pred flood as green, pred noFlood as red
plot_data.loc[plot_data.predicted_flood == True, 'c'] = 'green'
plot_data.loc[plot_data.predicted_flood == False, 'c'] = 'red'


#plot_data['c'] = 'red'
# plot_data.plot(x='created_at', y='flood_depth', color='blue', style='.')
plot_data['created_at'] = pd.to_datetime(plot_data.created_at, utc=True)

# utc -> Timestamp('2017-10-19 19:02:04.299000+0000', tz='UTC')
#plot_data[['created_at','flood_depth', 'c', 'predicted_flood']]

plot_data['num_index'] = range(0, plot_data.shape[0])
#plot_data = plot_data.set_index(['num_index'], append=True, drop=False)

plot_data.loc[155]
plot_data.iloc[0]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def clear_all_figures():
    for each in plt.get_fignums():
        plt.close(each)

#ml.score(features, labels, th, th0) # this is the number classified as positive

In [ ]:
plot_data.iloc[0,:]

In [ ]:
clear_all_figures()

fig = plt.figure()
path = plt.scatter(plot_data['created_at'], plot_data['flood_depth'], c=plot_data['c'], picker=True)
ax = plt.gca()
def onpick(event):
    import textwrap
    index = event.ind
    row = plot_data.iloc[index[0]]
    
    res_list = textwrap.wrap(row.text,40) 
    res_str = ''
    for each in res_list:
        res_str += each + '\n'
        
    ax.set_title(row.text)
    
    

    
plt.show()
fig.canvas.mpl_connect('pick_event', onpick)

In [ ]:
#for_projector = plot_data.copy()[:, ('text', 'predicted_flood')]
#
#copy_of_data = plot_data.copy()
#for_projector['label'] = copy_of_data['predicted_flood']
#
#res_tsv = for_projector.to_csv( sep='\t')
#
#for_projector

In [ ]:
th.shape
#vocab

In [ ]:
# here's how to see the weight of a particular word,
#index_of_water = vocab['water']
#th[index_of_water, :] # if this is positive it is correlated w/ flooding
#
#th[vocab['flooded'], :] # if this is positive it is correlated w/ flooding

how_pos = dict()
reverse_dict = dict()
for key, value in vocab.items():
    print(key, value)
    how_pos[key] = int(th[value, :])
    reverse_dict[value] = key

how_pos

#for each in 'Nearby water body is overflowing'.lower().split():
#    print(each + ' ' + str(how_pos[each]))
    
#for each in 'Spic Nagar rain water intrution from the velachery main road'.lower().split():
#    print(each + ' ' + str(how_pos[each]))
len(reverse_dict)
len(vocab)

In [ ]:

total_texts = loader.get_data()
total_texts['text']


In [ ]:
total_texts = loader.get_data()
df = pd.DataFrame(features.T, index=total_texts.index)
df.rename(columns=reverse_dict, inplace=True)
vectors_tab = df.to_csv(sep='\t', header=False, index=False)

v = open('vectors.tsv', 'w')
v.write(vectors_tab)
v.close()

def add_true_data(inp_df):
    flood = __get_flood_pkeys(start_known_flood, end_known_flood, config['database_engine'])
    no_flood = __get_no_flood_pkeys(start_known_flood, end_known_flood, config['database_engine'])
    for pkey in flood:
        inp_df.loc[ inp_df['pkey'] == pkey, 'true_flood'] = 1
    
    inp_df.loc[inp_df['true_flood'] != 1, 'true_flood'] = -1
    

meta_df = pd.DataFrame(df.index)

add_true_data(meta_df)

#meta_df = pd.DataFrame.merge(meta_df, )
no_index = total_texts.reset_index().drop('pkey', axis=1)

no_index['text'] = no_index['text'].str.replace('\n','')
meta_df = pd.concat([meta_df, no_index], axis=1, join='inner')
#meta_df = meta_df.join(total_texts)
#meta_df.append(total_texts)

metadata = meta_df.to_csv(sep='\t', index=False)

f = open('metadata.tsv', 'w')
f.write(metadata)
f.close()
#meta_df['pkey'] == 155
#meta_df.loc[ meta_df['pkey'] == 155] = 1
meta_df

In [ ]:
def find_top_and_bot_n_words(n, index_to_word, th):
#    if n < 0:
#        n = len(index_to_word)-n
    #partition = np.argpartition(th[:,0], [-n, n]) #[-n:]
    #top_n_indicies = partition[-n:]
    #bottom_n_indicies = partition[:n]
    partition = np.argsort(th[:,0], axis=0) #[-n:]
    top_n_indicies = partition[:n]
    bottom_n_indicies = partition[-n:]
    all_ind = np.hstack((top_n_indicies, bottom_n_indicies))
    res =  []
    for each in all_ind:
        res.append((reverse_dict[each], th[each]))
    return res
    
top = find_top_and_bot_n_words(200, reverse_dict, th)
for each in top:
    word, score = each
    occ = occur[word]
    freq = occ/len(occur)
    print(each, freq)

In [ ]:
a = np.argsort(th[:,0])
th[a]